<center>

# DECA: Detailed Expression Capture and Animation

Code in [![GitHub stars](https://img.shields.io/github/stars/yadiraf/DECA?style=social)](https://github.com/YadiraF/DECA)

Page at [![](https://img.shields.io/badge/Project-Page-blue?style=flat&logo=Google%20chrome&logoColor=blue)](https://deca.is.tue.mpg.de/)

Made by [![Yao](https://img.shields.io/twitter/follow/yaofeng1995?style=social)](https://twitter.com/yaofeng1995)

</center>

# **安装环境**

In [ ]:
# 克隆git仓库（Done）
# !cd C://Users//Administrator//Desktop
# !git clone https://github.com/Jimmy142857/DECA_Analyze.git

In [ ]:
#  安装库(需要GPU)

# !pip install -q gdown matplotlib
# !pip install --upgrade ipykernel
# !pip install numpy==1.23.5
# !pip install scipy chumpy scikit-image opencv-python
# !pip install PyYAML face-alignment yacs kornia ninja fvcore

In [ ]:
# 下载模型 (Done)

# print("Downloading FLAME2020 model...")              # 下载FLAME模型
# !gdown --id 18eHkbq2W3aJJVUNTM3QwFK0dPaeNoRAz -O FLAME2020.zip
# !unzip -o FLAME2020.zip -d data/

# print("Downloading deca_model...")                   # 下载训练好的DECA参数
# !gdown --id 1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje -O data/deca_model.tar

# **开始,需要GPU**

In [ ]:
!nvidia-smi

# **基本 Reconstruction**


In [ ]:
# 运行指定目录中的图片

print("Running experiments...")
import os
input_folder = "TestSamples/AFLW2000"
output_folder = os.path.join(input_folder, "results")

!python demos/demo_reconstruct.py -i $input_folder -s $output_folder --saveObj True

print(f'Please check all results in {output_folder}')

In [ ]:
# 运行单张图片(图片大小需要限制, 4M左右为上限)

print("Running experiments...")
import os

input_image = "C:/Users/Administrator/Desktop/DECA_Analyze/TestSamples/eyeClosed.jpg"
image_directory = os.path.dirname(input_image)
file_name = os.path.splitext(os.path.basename(input_image))[0]
output_folder = os.path.join(image_directory, file_name)

!python demos/demo_reconstruct.py -i $input_image -s $output_folder --saveObj True --saveImages True --useTex True

print(f'Please check all results in {output_folder}')

In [ ]:
# show results 
print('visualize one exmaple below')
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image

vispath_list = glob(output_folder+'/*_size.jpg')
for vispath in vispath_list:
  image = Image.open(vispath)
  plt.figure(figsize=(20, 20))
  plt.imshow(image)
  plt.axis("off")
  break
print(f'Please check all results in {output_folder}')


# **Expression Transfer**

In [ ]:
# 表情迁移
image_path = '/content/drive/MyDrive/Face_jimmy_abnormal/jimmy_1.jpg'
exp_path = '/content/drive/MyDrive/DECA/TestSamples/exp/7.jpg'
savefolder = ''

# --image_path $image_path --exp_path $exp_path

!python demos/demo_transfer.py --saveObj true --image_path $image_path --exp_path $exp_path

# **Reposing and Animation**

In [ ]:
#@title Reposing and Animation (Error 缺少文件FLAME_albedo_from_BFM.npz)
!python demos/demo_teaser.py 

# **Trainning!!!**

In [ ]:
#@title 尝试训练（Face Segmentation 步骤未完成）
# %cd /content/drive/MyDrive/DECA

# !python main_train.py --cfg configs/release_version/deca_pretrain.yml # 使用已有的预训练模型
!python main_train.py --cfg configs/release_version/deca_coarse.yml 
# !python main_train.py --cfg configs/release_version/deca_detail.yml 

# **代码解读**

In [ ]:
# DECA输出的68个landmarks查看
import cv2
import math
import numpy as np
from collections import defaultdict
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
%matplotlib inline
import matplotlib.pyplot as plt


file_path = 'palsy_1\\palsy_1_kpt3d.txt'

lmk = np.loadtxt(file_path, delimiter=' ')
lmk = lmk[:,:3]                              # 取前三列

  
for count, value in enumerate(np.arange(len(lmk))):
    plt.scatter(lmk[count, 0], lmk[count, 1], s = 15, marker = '.', c = 'r')
    plt.annotate(value, (lmk[count, 0], lmk[count, 1]))


plt.gca().invert_yaxis()      # Y轴需要翻转
plt.axis('off')
plt.show()

In [ ]:
# NoW 人脸框查看
import cv2
import numpy as np
from matplotlib import pyplot as plt

input_img = 'C:\\Users\\Administrator\\Desktop\\NoW\\NoW_Dataset\\final_release_version\\iphone_pictures\\1-25\\FaMoS_180625_03309_TA\\multiview_occlusions\\IMG_1357.jpg'
face_box = 'C:\\Users\\Administrator\\Desktop\\NoW\\NoW_Dataset\\final_release_version\\detected_face\\FaMoS_180625_03309_TA\\multiview_occlusions\\IMG_1357.npy'
save_pth = 'C:\\Users\\Administrator\\Desktop\\1357.jpg'

bbox = np.load(face_box, allow_pickle= True, encoding = 'latin1')
top = bbox.item()['top']
right = bbox.item()['right']
bottom = bbox.item()['bottom']
left = bbox.item()['left']

img = cv2.imread(input_img)
face_dected = img[top : bottom,left : right]
plt.imshow(face_dected)

cv2.imwrite(save_pth, face_dected)


In [ ]:
# decalib.datadet 输出查看
from decalib.datasets import datasets
from tqdm import tqdm

tes_pth = 'TestSamples\AFLW2000'
testdata = datasets.TestData(testpath = tes_pth, iscrop=True, crop_size=224, scale=1.25, face_detector='fan', sample_step=10)

for i in tqdm(range(len(testdata))):
    name = testdata[i]['imagename']
    images = testdata[i]['image'].to('cuda')[None,...]

In [ ]:
# decalib.now 输出查看
from decalib.datasets import now
from tqdm import tqdm

nowdata = now.NoWDataset()
for i in tqdm(range(len(nowdata))):
    name = nowdata[i]['imagename']
    images = nowdata[i]['image'].to('cuda')[None,...]

In [ ]:
# yaml为配置文件同yac库一起使用
import yaml
f = open(r'/content/drive/MyDrive/DECA/configs/release_version/deca_pretrain.yml')
y = yaml.load(f, Loader=yaml.FullLoader)
print (y.keys())

In [ ]:
# mask模板操作
from skimage.io import imread
import numpy as np
import torch

face_eye_mask_path = '/content/drive/MyDrive/DECA/data/uv_face_eye_mask.png'
mask = imread(face_eye_mask_path).astype(np.float32)/255.
mask = torch.from_numpy(mask[:,:,0])[None,None,:,:].contiguous()

In [ ]:
# displacement map查看
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

# np.set_printoptions(threshold=np.inf)
displacement_path = '/content/drive/MyDrive/DECA/data/fixed_displacement_256.npy'
disp = np.load(displacement_path)
disp = disp

imshow(disp, cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
# 密集网格模板查看，转为txt保存
import torch
import numpy as np

np.set_printoptions(threshold=np.inf)  # 解决输出不完整问题
dense_template_path = '/content/drive/MyDrive/DECA/data/texture_data_256.npy'
dense_txt = '/content/drive/MyDrive/DECA/data/texture_data_256.txt'

dense_template = np.load(dense_template_path, allow_pickle=True, encoding='latin1').item()
# with open(dense_txt, 'w') as dense:
#   for k,v in dense_template.items():
#     dense.write(k + ':' + str(v))
#     dense.write('\n')

print(type(dense_template))

In [ ]:
# 六参数维度保存

model = { 
  'n_shape':100,
  'n_tex':50,
  'n_exp' : 50,
  'n_cam': 3,
  'n_pose': 6,
  'n_light' : 27,
  }

param_list = ['shape', 'tex', 'exp', 'pose', 'cam', 'light']
param_dict = {i:model.get('n_' + i) for i in param_list}
print(param_dict)

In [ ]:
# 分析图片输入的预处理步骤
import numpy as np
from skimage.io import imread
from skimage.transform import estimate_transform

imagepath = 'C:\\Users\\Administrator\\Desktop\\NoW\\NoW_Dataset\\final_release_version\\iphone_pictures\\1-25\\FaMoS_180625_03309_TA\\multiview_occlusions\\IMG_1357.jpg'

image = np.array(imread(imagepath))
if len(image.shape) == 2:
    image = image[:,:,None].repeat(1,1,3)
if len(image.shape) == 3 and image.shape[2] > 3:
    image = image[:,:,:3]

h, w, _ = image.shape
src_pts = np.array([[0, 0], [0, h-1], [w-1, 0]])
DST_PTS = np.array([[0,0], [0, 223], [223, 0]])

tform = estimate_transform('similarity', src_pts, DST_PTS)

# **TEST**

In [ ]:
# 遍历文件夹  
import os 

g = os.walk("C:\\Users\\Administrator\\Desktop\\Now\\NoW_Dataset\\final_release_version\\iphone_pictures\\1-25")  

for path,dir_list,file_list in g:  
    for dir_name in dir_list:
        print(os.path.join(path, dir_name) )

In [ ]:
# 路径分隔
import os

path = "C:/Users/Administrator/Desktop/DECA_Analyze/TestSamples\jimmy\jimmy\jimmy.obj"

# 规范化路径并按照路径分隔符切割成列表
path_parts = os.path.normpath(path).split(os.path.sep)

print("Path parts:", path_parts)


In [ ]:
# 获取当前时间
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

print(current_time)

In [ ]:
# 中文路径图片读取
from pathlib import Path
import numpy as np
import cv2

file_path = 'C:\\Users\\Administrator\\Desktop\\DECA_Analyze\\TestSamples\\杰杰.jpg'

file_path = Path(file_path)

image = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

cv2.imshow('image', image)
cv2.waitKey(0)

In [ ]:
# 数据库查看
import sqlite3

def view_users_database(db_path='users.db'):
    try:
        # 连接到数据库
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 查询所有用户记录
        cursor.execute('SELECT * FROM users')
        rows = cursor.fetchall()

        if not rows:
            print("数据库中没有用户记录。")
        else:
            print("用户记录:")
            for row in rows:
                print(f"ID: {row[0]}, Username: {row[1]}, Age:{row[3]}, Gender:{row[4]}, Last_login_time:{row[5]},\n Login_count:{row[6]}, Created_at:{row[7]}, Password: {row[2]}")
                print()
    except sqlite3.Error as e:
        print(f"发生错误: {e}")

    finally:
        if conn:
            conn.close()

# 调用函数查看数据库
view_users_database()